# **Implementation of EM for Bayesian Networks.**

In [ ]:
!pip install pgmpy

In [20]:
!pip install networkx

In [21]:
import numpy as np
import networkx as nx
import pylab as plt
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD


In [22]:

model1 = BayesianNetwork([('A', 'B'), ('A', 'C'), ('B', 'D'), ('C', 'D')])
cpd_a = TabularCPD(variable='A', variable_card=2, values=[[0.5], [0.5]])
cpd_b = TabularCPD(variable='B', variable_card=2, values=[[0.3, 0.6],
[0.7, 0.4]], evidence=['A'], evidence_card=[2])
cpd_c = TabularCPD(variable='C', variable_card=2, values=[[0.4, 0.7],
[0.6, 0.3]], evidence=['A'], evidence_card=[2])
cpd_d = TabularCPD(variable='D', variable_card=2, values=[[0.2, 0.7,
0.6, 0.1], [0.8, 0.3, 0.4, 0.9]], evidence=['B', 'C'], evidence_card=[2,
2])

In [ ]:
#from pgmpy.models import BayesianModel
#model = BayesianModel([('A', 'B'), ('B', 'C'), ('A', 'D'), ('D', 'C')])

nx_graph = nx.DiGraph(model1.edges())
nx.draw(nx_graph, with_labels=True)
plt.show()

In [3]:
print(model1)
model1.nodes()

BayesianNetwork with 4 nodes and 4 edges


NodeView(('A', 'B', 'C', 'D'))

In [24]:
model1.add_cpds(cpd_a, cpd_b, cpd_c, cpd_d)
print(model1.get_cpds())
#len(model1.get_cpds())

[<TabularCPD representing P(A:2) at 0x7e8d08c95a20>, <TabularCPD representing P(B:2 | A:2) at 0x7e8d08c96110>, <TabularCPD representing P(C:2 | A:2) at 0x7e8d08c96770>, <TabularCPD representing P(D:2 | B:2, C:2) at 0x7e8d08c95f90>]


In [25]:
model1.local_independencies(['A', 'B', 'C', 'D'])

(B ⟂ C | A)
(C ⟂ B | A)
(D ⟂ A | C, B)

In [26]:
from pgmpy.sampling import BayesianModelSampling
samples = BayesianModelSampling(model1).forward_sample(size=int(1e5))
##max_iter = 100
#epsilon = 1e-6

  0%|          | 0/4 [00:00<?, ?it/s]

In [29]:
#learning para
from pgmpy.estimators import ExpectationMaximization
me = ExpectationMaximization(model1, samples)


In [28]:
me.get_parameters()

  0%|          | 0/100 [00:00<?, ?it/s]

[<TabularCPD representing P(A:2) at 0x7e8cfee58eb0>,
 <TabularCPD representing P(B:2 | A:2) at 0x7e8cfee594b0>,
 <TabularCPD representing P(C:2 | A:2) at 0x7e8cfee59570>,
 <TabularCPD representing P(D:2 | B:2, C:2) at 0x7e8cfee58f70>]

In [30]:
from pgmpy.estimators import ExpectationMaximization as EM

# Define a model structure with latent variables
model_latent = BayesianNetwork(
    ebunch=model1.edges(), latents=["C"]
)

# Dataset for latent model which doesn't have values for the latent variables
samples_latent = samples.drop(model_latent.latents, axis=1)

model_latent.fit(samples_latent, estimator=EM)

  0%|          | 0/100 [00:00<?, ?it/s]

In [31]:
print(model_latent.get_cpds("A"))

+------+---------+
| A(0) | 0.50392 |
+------+---------+
| A(1) | 0.49608 |
+------+---------+
